In [1]:
import pandas as pd
import numpy as np

### data loading

In [2]:
gene_list = ["SLC31A2","SPHK1","LYZ","PPIF","LST1","IL4I1","CXCL16","FAM26F","KYNU","IDO1","MXD1","IL1RN"]

In [3]:
database_path = "/Users/kang/Dropbox/Code/data/ToppGene_Data/"

In [13]:
df_cellcompartment = pd.read_csv(database_path + "GeneOntologyCellularComponent.txt", sep = "\t", header = 0, index_col = 0)

In [14]:
df_cellcompartment.head()

,symbol,concept_id,concept_name,concept_origin,url
gene_id,,,,,
387712,ENO4,GO:0000015,phosphopyruvate hydratase complex,,http://www.ebi.ac.uk/ego/DisplayGoTerm?id=GO:0...
2023,ENO1,GO:0000015,phosphopyruvate hydratase complex,,http://www.ebi.ac.uk/ego/DisplayGoTerm?id=GO:0...
2026,ENO2,GO:0000015,phosphopyruvate hydratase complex,,http://www.ebi.ac.uk/ego/DisplayGoTerm?id=GO:0...
2027,ENO3,GO:0000015,phosphopyruvate hydratase complex,,http://www.ebi.ac.uk/ego/DisplayGoTerm?id=GO:0...
1161,ERCC8,GO:0000109,nucleotide-excision repair complex,,http://www.ebi.ac.uk/ego/DisplayGoTerm?id=GO:0...


In [19]:
gene_list = np.intersect1d(np.unique(gene_list), np.unique(df_cellcompartment["symbol"]))
len(gene_list)

11

### get a pool

In [15]:
test = df_cellcompartment.copy()
test.set_index("symbol", inplace = True)

In [16]:
test.head()

,concept_id,concept_name,concept_origin,url
symbol,,,,
ENO4,GO:0000015,phosphopyruvate hydratase complex,,http://www.ebi.ac.uk/ego/DisplayGoTerm?id=GO:0...
ENO1,GO:0000015,phosphopyruvate hydratase complex,,http://www.ebi.ac.uk/ego/DisplayGoTerm?id=GO:0...
ENO2,GO:0000015,phosphopyruvate hydratase complex,,http://www.ebi.ac.uk/ego/DisplayGoTerm?id=GO:0...
ENO3,GO:0000015,phosphopyruvate hydratase complex,,http://www.ebi.ac.uk/ego/DisplayGoTerm?id=GO:0...
ERCC8,GO:0000109,nucleotide-excision repair complex,,http://www.ebi.ac.uk/ego/DisplayGoTerm?id=GO:0...


In [24]:
pool = np.unique(test.loc[gene_list,"concept_name"])
len(pool)

107

### enrichment for each concept

In [25]:
import gseapy as gp

In [ ]:
for concept in pool:
    ref = df_cellcompartment.loc[df_cellcompartment["concept_name"] == concept, "symbol"]
    

In [28]:
??gp.ssgsea